In [2]:
import numpy as np
from copy import deepcopy
from functools import reduce
from itertools import product
import networkx as nx
from matplotlib import pyplot as plt


In [12]:
class Node(object):
    
    
    def __init__(self, name, value):
        self.name = name
        self.value = value
        self.connections = dict()
    
    
    def set_connections(self, connections):
        for connection in connections:
            self.connections[connection[1]] = connection[0]
      
    
    def set_weight(self, connection):
        self.connections[connection[1]] = connection[0]
    
    
    def set_value(self, value):
        self.value = value
    
    
    def remove_connection(self, node):
        try:
            self.connections.pop(node)
        except KeyError:
            print("No connection detected")
    
    
    def __str__(self):
        return self.name
    
    
class Graph(object):
    
    
    def __init__(self, name):
        self.name = name
        self.nodes = []
        self.A = []
    
    
    def form_subgraphs(self, clusters):
        clusters_keys = list(clusters.keys())
        clusters_values  = list(clusters.values())
        subgraphs = []
        connections = []
        
        for cluster in clusters_keys:
            value = reduce(lambda x, y: x.value + y.value, clusters[cluster]) \
            if len(clusters[cluster]) > 1 else clusters[cluster][0].value 
            
            subgraphs.append(Node(cluster,value))    
        
        for i in range(len(clusters)):
            connections = []
            for j in range(len(clusters)):
                weight = 0
                for node_left in clusters_values[i]:
                    for node_right in node_left.connections.keys():
                        additive = node_left.connections[node_right] if node_right in clusters_values[j] \
                        and not node_right in clusters_values[i] else 0
                        weight += additive
                connections.append([weight, subgraphs[j]])
           
            subgraphs[i].set_connections(connections)
        
        aggregated = Graph('aggregated')
        aggregated.set_nodes(subgraphs)
        return aggregated
    
    
    def set_nodes(self, nodes):
        self.nodes = nodes
     
    
    def add_node(self, node):
        self.nodes.append(node)
        
        
    def remove_node(self, node):
        try:
            self.nodes.remove(node)
        except ValueError:
            print("no node " + str(node) + " in the graph")
    
    
    def __str__(self):
        return self.name
    
    
    def form_connection_matrix(self):
        self.A = np.zeros((len(self.nodes), len(self.nodes)))
        for i in range(len(self.nodes)):
            
            connection_nodes = list(self.nodes[i].connections.keys())
            for j in range(len(self.nodes)):
                self.A[i, j] = self.nodes[i].connections[self.nodes[j]] \
                if self.nodes[j] in connection_nodes else 0
        
        return self.A
    
    
    def update_values(self, nodes_values):
        for node in self.nodes:
            node.set_value(nodes_values[node])
    
    
    def send_impulses(self, impulses):
        previous = np.zeros((len(self.nodes), 1))
        current = np.array([[node.value] for node in self.nodes])
        for impulse in impulses:
            new = current + self.A.T @  (current - previous) + impulse
            nodes_values = {self.nodes[i]: current[i, 0]   for i in range(len(self.nodes))}
            previous = current
            current = new
            self.update_values(nodes_values)
        
        print(current)
        
    def search_cycles(self):
        cycles = set()
        for node in self.nodes:
            print(self.nodes.index(node))
            layer = 0
            vertexes = dict()
            vertexes[layer] = set([node])
            while layer < len(self.nodes):
                nexts = set()
                previous_layers = reduce(lambda x, y: x.union(y), list(vertexes.values())[: layer]) if layer > 0 else []
                for vertex in vertexes[layer]:
                    if vertex not in previous_layers:
                        for next_vertex in vertex.connections.keys():
                            nexts.add(next_vertex)
                    else:
                        nexts.add(vertex)
                
                layer += 1
                if nexts != set():
                    vertexes[layer] = nexts
                else:
                    break  
                
            #for i in range(layer):
            #    print(list(map(lambda val: str(val), vertexes[i])))
            #print("----------")
            
            for length in range(1, len(vertexes)):
                if node in vertexes[length]:
                    additive = list(product(*[vertexes[i] for i in range(0, length)]))
                    for cycle in additive:
                        cycles.add(tuple(set(cycle)))
        
        for cycle in cycles:
            print(list(map(lambda val: str(val), cycle)))
            print("----------")
                #cycle = set()
                #for layer in range(1, length):
                #    cycle.add()
            #for node in nod
                
        
        
        
                

In [4]:
A = Node('A', 1)

C = Node('C', 2)

B = Node('B', 3)

impulse = [np.array([[0], [0.1], [0.5]]) if i < 10 else np.array([[0], [0], [0]]) for i in range(50)]

A.set_connections([[0.8, C]])
B.set_connections([[0.2, B], [0.3, C], [0.111, A]])
C.set_connections([[0.2, C], [0, B], [0.3, A]])
Cognitive_Parasha = Graph('Parasha')
Cognitive_Parasha.set_nodes([A, B, C])
Cognitive_Parasha.form_subgraphs({'Bois': [A], 'Parasites': [B, C]})
Cognitive_Parasha.form_connection_matrix()
Cognitive_Parasha.search_cycles()
#Cognitive_Parasha.send_impulses(impulse)

['B']
----------
['C']
----------
['B', 'A', 'C']
----------
['C', 'B']
----------
['A', 'C']
----------
['A', 'B']
----------
['A', 'B', 'C']
----------
['B', 'C']
----------
['C', 'B', 'A']
----------


In [382]:
list(product(*[{'a'}, {'1', '2'}],))

[('a', '2'), ('a', '1')]

In [ ]:
nodes = []
for i in range(15):
    nodes.append(Node(str(i), 0))
    
for node in nodes:
    node.set_connections([[1, vertex] for vertex in nodes[6: 14]])
    
FullMap = Graph(nodes)
FullMap.set_nodes(nodes)
FullMap.search_cycles()

0
1
2
3
4
5
6
